In [61]:
import os

import dbf
from dbfread import DBF
import pandas as pd

In [62]:
pathfile = r"c:\ProgramData\AVEVA\Citect SCADA 2018 R2\User\Example"
# filenam = r"equip.DBF"
filenam = r"pgdynobj.DBF"
filename = os.path.join(pathfile,filenam)

cols = ['NAME', 'CLUSTER', 'TYPE', 'COMMENT', 'IODEVICE', 'TAGPREFIX']
values = ['tr1.P3T32', 'Cluster1', 'Loop', 'Тест', 'CICODE', 'tr1_open']
params = dict(zip(cols, values))

table = dbf.Table(filename, codepage='cp1251')
table.open(mode=dbf.READ_WRITE)
table.append(params)

table.close()

In [20]:
# Удаление строки
rec = table[-1]
dbf.delete(rec)
table.pack()

In [67]:
pathfile = r"c:\ProgramData\AVEVA\Citect SCADA 2018 R2\User\Example"
# filenam = r"equip.DBF"
filenam = r"variable.DBF"
filename = os.path.join(pathfile,filenam)

db_in_dbf = DBF(filename, encoding='cp1251')
df = pd.DataFrame(db_in_dbf )
df

,NAME,TYPE,UNIT,ADDR,RAW_ZERO,RAW_FULL,ENG_ZERO,ENG_FULL,ENG_UNITS,FORMAT,...,EQUIP,ITEM,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,CUSTOM5,CUSTOM6,CUSTOM7,CUSTOM8
0,Agitator_Cmd,DIGITAL,DISK_PLC,D302,,,,,,,...,,,,,,,,,,
1,Arrow_fiber_optical,DIGITAL,IODevInternal,D24,,,,,,,...,,,,,,,,,,
2,Arrow_Network3,DIGITAL,IODevInternal,D15,,,,,,,...,,,,,,,,,,
3,BATCH,INT,DISK_PLC,I9,0,32000,0,32000,,#####,...,,,,,,,,,,
4,BatchTotal,INT,DISK_PLC,I10,0,32000,0,32000,,#####,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,IOServer_Internal_DriverError,INT,CICODE,"IODeviceInfo(""Internal"",5)",,,,,,,...,IOServer.Internal,DriverError,,,,,,,,
1111,IOServer_Internal_Protocol,STRING,CICODE,"IODeviceInfo(""Internal"",1)",,,,,,,...,IOServer.Internal,Protocol,,,,,,,,
1112,IOServer_Internal_State,INT,CICODE,"IODeviceInfo(""Internal"",10)",,,,,,,...,IOServer.Internal,State,,,,,,,,
1113,IOServer_Internal_ReadTimeAvg,REAL,CICODE,"IODeviceInfo(""Internal"",9)",,,,,,,...,IOServer.Internal,ReadTimeAvg,,,,,,,,


In [68]:
df

,NAME,TYPE,UNIT,ADDR,RAW_ZERO,RAW_FULL,ENG_ZERO,ENG_FULL,ENG_UNITS,FORMAT,...,EQUIP,ITEM,CUSTOM1,CUSTOM2,CUSTOM3,CUSTOM4,CUSTOM5,CUSTOM6,CUSTOM7,CUSTOM8
0,Agitator_Cmd,DIGITAL,DISK_PLC,D302,,,,,,,...,,,,,,,,,,
1,Arrow_fiber_optical,DIGITAL,IODevInternal,D24,,,,,,,...,,,,,,,,,,
2,Arrow_Network3,DIGITAL,IODevInternal,D15,,,,,,,...,,,,,,,,,,
3,BATCH,INT,DISK_PLC,I9,0,32000,0,32000,,#####,...,,,,,,,,,,
4,BatchTotal,INT,DISK_PLC,I10,0,32000,0,32000,,#####,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,IOServer_Internal_DriverError,INT,CICODE,"IODeviceInfo(""Internal"",5)",,,,,,,...,IOServer.Internal,DriverError,,,,,,,,
1111,IOServer_Internal_Protocol,STRING,CICODE,"IODeviceInfo(""Internal"",1)",,,,,,,...,IOServer.Internal,Protocol,,,,,,,,
1112,IOServer_Internal_State,INT,CICODE,"IODeviceInfo(""Internal"",10)",,,,,,,...,IOServer.Internal,State,,,,,,,,
1113,IOServer_Internal_ReadTimeAvg,REAL,CICODE,"IODeviceInfo(""Internal"",9)",,,,,,,...,IOServer.Internal,ReadTimeAvg,,,,,,,,
